This project aims to use data from PowerCharge Utilities dataset to achieve the following aims for the company:
1. Conduct a comprehensive analysis to ascertain the current capacity of the company to handle increased load from EV charging stations
2. Identify potential vulnerabilities within the distribution network that may hinder reliable electricity delivery
3. Develop a data-driven strategy that maximizes effort, minimizes cost, and ensures grid reliability

First we import our libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Let us import the datasets. We will be working the address dataset, weather dataset, geospatial dataset and ev-distribution dataset.

import address dataset and set columns

In [2]:
address_headers = ["name", "surname", "address", "city", "city_code", "zipcode"]
address_df = pd.read_csv("addresses.csv", names = address_headers)
address_df.head()

,name,surname,address,city,city_code,zipcode
0,John,Doe,120 jefferson st.,Riverside,NJ,8075
1,Jack,McGinnis,220 hobo Av.,Phila,PA,9119
2,"John ""Da Man""",Repici,120 Jefferson St.,Riverside,NJ,8075
3,Stephen,Tyler,"7452 Terrace ""At the Plaza"" road",SomeTown,SD,91234
4,NaN,Blankman,NaN,SomeTown,SD,298


For data wrangling, we check for data type, missing values, inaccurate records, consistency of formats

In [3]:
address_df.dtypes

name         object
surname      object
address      object
city         object
city_code    object
zipcode       int64
dtype: object

In [4]:
for column in address_df.columns.values.tolist():
    address_empty = address_df[column].isnull()
    print(address_empty.value_counts())

name
False    5
True     1
Name: count, dtype: int64
surname
False    6
Name: count, dtype: int64
address
False    5
True     1
Name: count, dtype: int64
city
False    6
Name: count, dtype: int64
city_code
False    6
Name: count, dtype: int64
zipcode
False    6
Name: count, dtype: int64


there is only missing value in name and address. From the dataset, we see that both missing values are on the same row, so we delete the entire row.

In [5]:
address_df.dropna(axis=0, inplace = True)

In [6]:
address_df.reset_index()

,index,name,surname,address,city,city_code,zipcode
0,0,John,Doe,120 jefferson st.,Riverside,NJ,8075
1,1,Jack,McGinnis,220 hobo Av.,Phila,PA,9119
2,2,"John ""Da Man""",Repici,120 Jefferson St.,Riverside,NJ,8075
3,3,Stephen,Tyler,"7452 Terrace ""At the Plaza"" road",SomeTown,SD,91234
4,5,"Joan ""the bone"", Anne",Jet,"9th, at Terrace plc",Desert City,CO,123


we are done with cleaning this dataset, except for removing the wrong index. we'll get back to it. Again, the dataset does not appear to be useful.

weather dataset wrangling, as usual, check the data type, and missing data. We will need to create the year, month, day and time from the datetime. 

In [7]:
weather_df = pd.read_csv("synthetic_weather_data.csv")
weather_df.head()

,Timestamp,Temperature (°C),Precipitation (mm),Weather_Conditions,Substation_ID
0,2020-01-03 11:00:00,28.865989,0.313611,Snowy,Substation_744
1,2020-01-03 17:00:00,28.902260,0.817728,Clear,Substation_516
2,2020-01-04 04:00:00,24.212651,4.516506,Clear,Substation_261
3,2020-01-04 23:00:00,29.024606,3.821278,Snowy,Substation_560
4,2020-01-05 00:00:00,24.126655,4.965771,Snowy,Substation_258


In [8]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Timestamp           97 non-null     object 
 1   Temperature (°C)    97 non-null     float64
 2   Precipitation (mm)  97 non-null     float64
 3   Weather_Conditions  97 non-null     object 
 4   Substation_ID       97 non-null     object 
dtypes: float64(2), object(3)
memory usage: 3.9+ KB


In [9]:
weather_df.dtypes

Timestamp              object
Temperature (°C)      float64
Precipitation (mm)    float64
Weather_Conditions     object
Substation_ID          object
dtype: object

In [10]:
for column in weather_df.columns.values.tolist():
    weather_empty = weather_df[column].isnull()
    print(weather_empty.value_counts())

Timestamp
False    97
Name: count, dtype: int64
Temperature (°C)
False    97
Name: count, dtype: int64
Precipitation (mm)
False    97
Name: count, dtype: int64
Weather_Conditions
False    97
Name: count, dtype: int64
Substation_ID
False    97
Name: count, dtype: int64


there are no missing values for weather. let us expand datetime

In [11]:
from datetime import date

In [12]:
weather_df["Timestamp"] = pd.to_datetime(weather_df["Timestamp"])

In [13]:
weather_df.dtypes

Timestamp             datetime64[ns]
Temperature (°C)             float64
Precipitation (mm)           float64
Weather_Conditions            object
Substation_ID                 object
dtype: object

In [14]:
weather_df['year'] = weather_df['Timestamp'].dt.year

In [15]:
weather_df['month'] = weather_df['Timestamp'].dt.month_name()
weather_df['day'] = weather_df['Timestamp'].dt.day_name()

In [16]:
weather_df['hour'] = weather_df['Timestamp'].dt.hour

In [17]:
weather_df

,Timestamp,Temperature (°C),Precipitation (mm),Weather_Conditions,Substation_ID,year,month,day,hour
0,2020-01-03 11:00:00,28.865989,0.313611,Snowy,Substation_744,2020,January,Friday,11
1,2020-01-03 17:00:00,28.902260,0.817728,Clear,Substation_516,2020,January,Friday,17
2,2020-01-04 04:00:00,24.212651,4.516506,Clear,Substation_261,2020,January,Saturday,4
3,2020-01-04 23:00:00,29.024606,3.821278,Snowy,Substation_560,2020,January,Saturday,23
4,2020-01-05 00:00:00,24.126655,4.965771,Snowy,Substation_258,2020,January,Sunday,0
...,...,...,...,...,...,...,...,...,...
92,2020-02-10 10:00:00,19.379869,0.957467,Clear,Substation_562,2020,February,Monday,10
93,2020-02-10 13:00:00,19.001256,3.359719,Clear,Substation_562,2020,February,Monday,13
94,2020-02-10 14:00:00,23.231818,2.297417,Clear,Substation_74,2020,February,Monday,14
95,2020-02-11 02:00:00,31.228543,2.688862,Rainy,Substation_365,2020,February,Tuesday,2


In [18]:
weather_df.describe(include = "all")

,Timestamp,Temperature (°C),Precipitation (mm),Weather_Conditions,Substation_ID,year,month,day,hour
count,97,97.000000,97.000000,97,97,97.0,97,97,97.000000
unique,NaN,NaN,NaN,3,49,NaN,2,7,NaN
top,NaN,NaN,NaN,Rainy,Substation_560,NaN,January,Monday,NaN
freq,NaN,NaN,NaN,39,4,NaN,69,17,NaN
mean,2020-01-23 05:32:09.896907520,25.311668,2.542096,NaN,NaN,2020.0,NaN,NaN,12.216495
min,2020-01-03 11:00:00,12.449425,0.007923,NaN,NaN,2020.0,NaN,NaN,0.000000
25%,2020-01-13 11:00:00,22.245088,1.390244,NaN,NaN,2020.0,NaN,NaN,6.000000
50%,2020-01-22 17:00:00,25.248302,2.688862,NaN,NaN,2020.0,NaN,NaN,12.000000
75%,2020-02-03 19:00:00,28.902260,3.656106,NaN,NaN,2020.0,NaN,NaN,18.000000
max,2020-02-11 06:00:00,34.104493,4.965771,NaN,NaN,2020.0,NaN,NaN,23.000000


to clean the geospatial dataset, we check for data type, missing values and others. Transmission line capacity is the amount of electricity that can be transmitted along that line.

In [19]:
geospatial_df = pd.read_csv("synthetic_geospatial_data.csv")
geospatial_df.head()

,Substation_ID,Substation_Location,Transmission_Line_Capacity (MW)
0,Substation_907,"(57.25813554648914, -70.90582041775649)",40
1,Substation_218,"(70.02538473360673, -84.38057237161145)",71
2,Substation_339,"(80.39323542610704, -75.79784143914193)",85
3,Substation_773,"(49.12307771334813, -87.3598904388042)",10
4,Substation_349,"(66.79887060665365, -86.51124108848195)",40


In [20]:
geospatial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Substation_ID                    50 non-null     object
 1   Substation_Location              50 non-null     object
 2   Transmission_Line_Capacity (MW)  50 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


In [21]:
geospatial_df.dtypes

Substation_ID                      object
Substation_Location                object
Transmission_Line_Capacity (MW)     int64
dtype: object

In [22]:
geospatial_df.describe(include = "all")

,Substation_ID,Substation_Location,Transmission_Line_Capacity (MW)
count,50,50,50.000000
unique,49,50,NaN
top,Substation_11,"(57.25813554648914, -70.90582041775649)",NaN
freq,2,1,NaN
mean,NaN,NaN,52.740000
std,NaN,NaN,24.857808
min,NaN,NaN,10.000000
25%,NaN,NaN,33.250000
50%,NaN,NaN,51.500000
75%,NaN,NaN,74.750000


In [23]:
geospatial_df['Substation_ID'].value_counts()

Substation_ID
Substation_11     2
Substation_907    1
Substation_133    1
Substation_365    1
Substation_764    1
Substation_422    1
Substation_240    1
Substation_720    1
Substation_523    1
Substation_516    1
Substation_845    1
Substation_911    1
Substation_54     1
Substation_816    1
Substation_139    1
Substation_744    1
Substation_890    1
Substation_258    1
Substation_152    1
Substation_74     1
Substation_717    1
Substation_64     1
Substation_208    1
Substation_952    1
Substation_711    1
Substation_978    1
Substation_218    1
Substation_261    1
Substation_339    1
Substation_773    1
Substation_349    1
Substation_752    1
Substation_294    1
Substation_301    1
Substation_443    1
Substation_996    1
Substation_686    1
Substation_999    1
Substation_560    1
Substation_562    1
Substation_87     1
Substation_546    1
Substation_321    1
Substation_805    1
Substation_628    1
Substation_981    1
Substation_659    1
Substation_93     1
Substation_470    1
Name: 

Let us wrangle ev_distribution data

In [24]:
ev_distribution_df = pd.read_csv("synthetic_ev_distribution_data.csv")
ev_distribution_df.head()

,Timestamp,Geographical_Area,Customer_Type,Electricity_Consumption (kWh),EV_Charging_Station_Location,EV_Charging_Station_Specifications,EV_Type,Charging_Habit,Number_of_EVs,Substation_ID
0,2020-01-10 18:00:00,Urban,Industrial,714.394409,"(64.01943603679787, -129.53196438913122)",Type 2,Electric Car,Occasional,8,Substation_907
1,2020-01-22 17:00:00,Rural,Residential,554.336019,"(48.447826648452946, -71.98859080823637)",Type 2,Electric Scooter,Weekly,5,Substation_907
2,2020-01-10 18:00:00,Urban,Industrial,714.394409,"(64.01943603679787, -129.53196438913122)",Type 2,Electric Car,Occasional,8,Substation_907
3,2020-01-22 17:00:00,Rural,Residential,554.336019,"(48.447826648452946, -71.98859080823637)",Type 2,Electric Scooter,Weekly,5,Substation_907
4,2020-01-10 18:00:00,Urban,Industrial,714.394409,"(64.01943603679787, -129.53196438913122)",Type 2,Electric Car,Occasional,8,Substation_907


In [25]:
ev_distribution_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Timestamp                           500 non-null    object 
 1   Geographical_Area                   500 non-null    object 
 2   Customer_Type                       500 non-null    object 
 3   Electricity_Consumption (kWh)       500 non-null    float64
 4   EV_Charging_Station_Location        500 non-null    object 
 5   EV_Charging_Station_Specifications  500 non-null    object 
 6   EV_Type                             500 non-null    object 
 7   Charging_Habit                      500 non-null    object 
 8   Number_of_EVs                       500 non-null    int64  
 9   Substation_ID                       500 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 39.2+ KB


In [26]:
for column in ev_distribution_df.columns.values.tolist():
    empty_ev = ev_distribution_df[column].isnull()
    print(empty_ev.value_counts())

Timestamp
False    500
Name: count, dtype: int64
Geographical_Area
False    500
Name: count, dtype: int64
Customer_Type
False    500
Name: count, dtype: int64
Electricity_Consumption (kWh)
False    500
Name: count, dtype: int64
EV_Charging_Station_Location
False    500
Name: count, dtype: int64
EV_Charging_Station_Specifications
False    500
Name: count, dtype: int64
EV_Type
False    500
Name: count, dtype: int64
Charging_Habit
False    500
Name: count, dtype: int64
Number_of_EVs
False    500
Name: count, dtype: int64
Substation_ID
False    500
Name: count, dtype: int64


no missing data

In [27]:
ev_distribution_df.describe(include="all")

,Timestamp,Geographical_Area,Customer_Type,Electricity_Consumption (kWh),EV_Charging_Station_Location,EV_Charging_Station_Specifications,EV_Type,Charging_Habit,Number_of_EVs,Substation_ID
count,500,500,500,500.000000,500,500,500,500,500.000000,500
unique,97,3,3,NaN,97,3,3,3,NaN,49
top,2020-01-14 07:00:00,Rural,Commercial,NaN,"(31.57926468770396, -112.34086442203312)",Type 2,Electric Scooter,Daily,NaN,Substation_11
freq,14,237,183,NaN,14,235,227,178,NaN,20
mean,NaN,NaN,NaN,506.188554,NaN,NaN,NaN,NaN,5.240000,NaN
std,NaN,NaN,NaN,80.251238,NaN,NaN,NaN,NaN,2.577535,NaN
min,NaN,NaN,NaN,315.912577,NaN,NaN,NaN,NaN,1.000000,NaN
25%,NaN,NaN,NaN,439.978312,NaN,NaN,NaN,NaN,3.000000,NaN
50%,NaN,NaN,NaN,505.820872,NaN,NaN,NaN,NaN,5.000000,NaN
75%,NaN,NaN,NaN,557.230712,NaN,NaN,NaN,NaN,8.000000,NaN


There are 3 unique geographical area

3 unique customer types

97 unique ev-charging-station-location

3 unique ev-charging-station-specification

3 unique ev-type

3 unique charging habit

49 unique substation ID

97 unique Timestamp 

We now have to merge the datasets into one.

In [29]:
ev_distribution_df

,Timestamp,Geographical_Area,Customer_Type,Electricity_Consumption (kWh),EV_Charging_Station_Location,EV_Charging_Station_Specifications,EV_Type,Charging_Habit,Number_of_EVs,Substation_ID
0,2020-01-10 18:00:00,Urban,Industrial,714.394409,"(64.01943603679787, -129.53196438913122)",Type 2,Electric Car,Occasional,8,Substation_907
1,2020-01-22 17:00:00,Rural,Residential,554.336019,"(48.447826648452946, -71.98859080823637)",Type 2,Electric Scooter,Weekly,5,Substation_907
2,2020-01-10 18:00:00,Urban,Industrial,714.394409,"(64.01943603679787, -129.53196438913122)",Type 2,Electric Car,Occasional,8,Substation_907
3,2020-01-22 17:00:00,Rural,Residential,554.336019,"(48.447826648452946, -71.98859080823637)",Type 2,Electric Scooter,Weekly,5,Substation_907
4,2020-01-10 18:00:00,Urban,Industrial,714.394409,"(64.01943603679787, -129.53196438913122)",Type 2,Electric Car,Occasional,8,Substation_907
...,...,...,...,...,...,...,...,...,...,...
495,2020-01-15 12:00:00,Urban,Residential,428.155578,"(21.013012971309866, -89.33001011359114)",Type 2,Electric Scooter,Occasional,8,Substation_470
496,2020-01-15 12:00:00,Urban,Residential,428.155578,"(21.013012971309866, -89.33001011359114)",Type 2,Electric Scooter,Occasional,8,Substation_470
497,2020-02-04 21:00:00,Urban,Residential,400.164596,"(59.359457886399674, -135.5756163019498)",DC Fast,Electric Bike,Weekly,9,Substation_470
498,2020-01-15 12:00:00,Urban,Residential,428.155578,"(21.013012971309866, -89.33001011359114)",Type 2,Electric Scooter,Occasional,8,Substation_470


transfer the clean data to another dataset for analysis. comment out the codes.

In [40]:
#weather_df.to_csv("weather.csv")

In [39]:
#geospatial_df.to_csv("geospatial.csv")

In [38]:
#ev_distribution_df.to_csv("ev_distribution.csv")